In [2]:
#we're using the venv from deeplabv3plus and just pip install scikit-image on top of it

import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from skimage.measure import label

In [3]:
def create_coco_xml(boxes, file_name, folder, image_shape):
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = folder
    ET.SubElement(root, "filename").text = file_name
    ET.SubElement(root, "path").text = os.path.join(folder, file_name)
    
    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(image_shape[1])
    ET.SubElement(size, "height").text = str(image_shape[0])
    ET.SubElement(size, "depth").text = str(image_shape[2] if len(image_shape) > 2 else 1)

    ET.SubElement(root, "segmented").text = "0"

    for box in boxes:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = "disease"
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"

        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(box[1])
        ET.SubElement(bndbox, "ymin").text = str(box[0])
        ET.SubElement(bndbox, "xmax").text = str(box[3])
        ET.SubElement(bndbox, "ymax").text = str(box[2])

    tree = ET.ElementTree(root)
    return tree

In [4]:
def extract_bboxes(mask):
    labeled_mask = label(mask)
    boxes = []
    for region in range(1, np.max(labeled_mask) + 1):
        where = np.where(labeled_mask == region)
        y1, x1, y2, x2 = np.min(where[0]), np.min(where[1]), np.max(where[0]), np.max(where[1])
        boxes.append([y1, x1, y2 + 1, x2 + 1])  # Adjust indices as needed
    return boxes

In [5]:
def process_masks(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for mask_filename in os.listdir(input_folder):
        if mask_filename.endswith('.png'):  # Assuming masks are PNG files
            mask_path = os.path.join(input_folder, mask_filename)
            mask = cv2.imread(mask_path, 0)  # Load as grayscale
            boxes = extract_bboxes(mask > 0)  # Threshold to create binary mask if needed
            
            tree = create_coco_xml(boxes, mask_filename, output_folder, mask.shape)
            output_path = os.path.join(output_folder, os.path.splitext(mask_filename)[0] + '.xml')
            tree.write(output_path)

In [6]:
# Example usage:
input_masks_folder = './masks'
output_xml_folder = './annots'
process_masks(input_masks_folder, output_xml_folder)